In [4]:
import math

In [3]:

def swapV2(x, r1, r2, swapfee=0.003, useFloat=False):
	if useFloat:
		# As mathematical expression
		return r2 - r1*r2/(r1 + (1-swapfee)*x)
	else:
		# Copy Uniswap V2 code:
		# if type(x) != int or type(r1) != int or type(r2) != int:
		# 	print("Warning, non-integer input")
		amountInWithFee = x * 997

		numerator = amountInWithFee * r2
		
		denominator = r1 * 1000
		denominator += amountInWithFee
		amountOut = numerator // denominator
		return amountOut

In [2]:

def tickFromSqrtPrice(sqrtPrice):
    # Calculated tick from SqrtPrice:
    calculatedTick = math.log(sqrtPrice)/math.log(math.sqrt(1.0001))
    calculatedTickFloor = math.floor(calculatedTick) 
    return calculatedTickFloor

def sqrtPriceFromTick(currentTick):
    sqrtPrice = math.sqrt(1.0001 ** currentTick)
    return sqrtPrice

def swapV3(sqrtPriceX96,liquidity,fee,spacing,inputAmount,zeroForOne):
    din = inputAmount - inputAmount*fee//10**6
    q96 = 2**96
    curTick = tickFromSqrtPrice(sqrtPriceX96)
    print("curTick", curTick)
    leftTick = (curTick//spacing)*spacing
    print("leftTick", leftTick)
    rightTick = leftTick + spacing
    print("rightTick", rightTick)

    leftPrice = sqrtPriceFromTick(leftTick)
    print("leftPrice", leftPrice)
    rightPrice = sqrtPriceFromTick(rightTick)
    print("rightPrice", rightPrice)


    if zeroForOne:
        num = liquidity*2**96
        product = din*sqrtPriceX96
        den = num + product
        price_next = math.ceil(num*sqrtPriceX96/den)
        print("price_next zfo", price_next)
        
        if price_next < leftPrice:
            price_next = leftPrice
    else:
        price_next = sqrtPriceX96 + q96*din//liquidity
        print("price_next else", price_next)

        if price_next > rightPrice:
            price_next = rightPrice

    print("price_next", price_next)
    print("sqrtPriceX96", sqrtPriceX96)

    spax,spbx = (price_next,sqrtPriceX96) if zeroForOne else (sqrtPriceX96,price_next)
    dx = ((liquidity*2**96*(spbx-spax))//spbx)//spax
    print("dx", dx)
    dy = (liquidity*(spbx-spax))//2**96
    print("dy", dy)
    # return (dx,dy)
    if zeroForOne:
        return (dx,dy)
    else:
        return (dy,dx)

In [ ]:
# Pool V2:
# zeroForOne = True
# inAmount = 2722768
# outAmount = 747811405678518267894
# reserve0 = 21680170
# reserve1 = 6720212576627902105169
res = swapV2(2722768, 21680170, 6720212576627902105169, swapfee=0.003, useFloat=False)
print(res)

# Pool V3:
# zeroForOne = False
# inAmount = 747811405678518267894
# outAmount = 35066737294955491085
# sqrtPriceX96 = 362221472736634002846898497940
# liquidity = 16113270320552177866762
# fee = 10000
# spacing = 200
res = swapV3(362221472736634002846898497940, 16113270320552177866762, 10000, 200, 747811405678518267894, False)
print(res)

# Pool V3:
# zeroForOne = True
# inAmount = 35066737294955491085
# outAmount = 17892182
# sqrtPriceX96 = 56679045957199870779219
# liquidity = 829079219992681553

In [8]:
# Pool V2:
# zeroForOne = False
# inAmount = 23679110
# outAmount = 45588582061590989869
# reserve0 = 1742956820743678908071922
# reserve1 = 902567554539
res = swapV2(23679110, 902567554539, 1742956820743678908071922, swapfee=0.003, useFloat=False)
print(res) # 45588582061590989869

# Pool V2:
# zeroForOne = True
# inAmount = 45588582061590989869
# outAmount = 135874862302335845816944797
# reserve0 = 139435156344542725977
# reserve1 = 552706007991834336427518147
res = swapV2(45588582061590989869, 139435156344542725977, 552706007991834336427518147, swapfee=0.003, useFloat=False)
print(res) # 135874862302335845816944797

# Pool V2:
# zeroForOne = False
# inAmount = 135874862302335845816944797
# outAmount = 33559548
# reserve0 = 532116425
# reserve1 = 2012486009208605494325294140
res = swapV2(135874862302335845816944797, 2012486009208605494325294140, 532116425, swapfee=0.003, useFloat=False)
print(res) # 33559548



45588582061590989869
135874862302335845816944797
33559548
